In [1]:
from __future__ import division, print_function

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from glob import glob
from imageio import imread
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

torch.cuda.device_count()

9L

In [2]:
print(torch.cuda.is_available())

True


In [3]:
# class TableDataset(Dataset):
#     """The training table dataset.
#     """
#     def __init__(self, x_path=None, y_path=None):
#         if x_path is None or y_path is None:
#             raise ValueError("No data source specified.")
        
#         x_filenames = glob(x_path + '*.png')
#         y_filenames = glob(y_path + '*.png')
        
#         self.x_data = [torch.from_numpy(self.normalize_image(imread(filename)).transpose(2, 0, 1)) for filename in x_filenames]
#         self.y_data = [torch.from_numpy(self.normalize_image(imread(filename)).reshape(1, *imread(filename).shape)) for filename in y_filenames]
#         self.len = len(self.x_data)
        
#     def __getitem__(self, index):
#         return self.x_data[index], self.y_data[index]
        
#     def __len__(self):
#         return self.len
    
#     def normalize_image(self, img):
#         return img / 255
        

In [4]:
# dataset = TableDataset('/home/calvinku/projects/yangzhe/data/cell01/',
#                        '/home/calvinku/projects/yangzhe/data/xu_label_cell01/')

# train_loader = DataLoader(dataset=dataset,
#                          batch_size=1,
#                          shuffle=True,
#                          num_workers=1)

In [ ]:
img = imread('/home/calvinku/projects/yangzhe/data/cell01/1.png')
label = imread('/home/calvinku/projects/yangzhe/data/xu_label_cell01/1.png')

print(img.shape, label.shape)

In [ ]:
print(img)

In [ ]:
print(label)

In [ ]:
plt.figure(figsize=(16, 9))
plt.imshow(img)
plt.show()

plt.figure(figsize=(16, 9))
plt.imshow(label, cmap='Greys')
plt.show()

In [5]:
class FCN(nn.Module):

    def __init__(self):
        super(FCN, self).__init__()
        self.dropout = nn.Dropout2d(p=0.5)
            
        self.conv1 = nn.DataParallel(nn.Conv2d(3, 64, 3, stride=1, padding=0))
        self.pool1 = nn.MaxPool2d(2, stride=1)
        self.relu1 = nn.ELU()
        
        self.conv2 = nn.DataParallel(nn.Conv2d(64, 128, 3, stride=1, padding=0))
        self.pool2 = nn.MaxPool2d(2, stride=1)
        self.relu2 = nn.ELU()
        
        self.conv3 = nn.Conv2d(128, 512, 3, stride=1, padding=0)
        self.pool3 = nn.MaxPool2d(2, stride=1)
        self.relu3 = nn.ELU()
                
        self.deconv11 = nn.ConvTranspose2d(512, 128, kernel_size=3, stride=1, padding=0)
        self.deconv12 = nn.ConvTranspose2d(128, 128, kernel_size=2, stride=1, padding=0)
        self.relu4 = nn.ELU()
        
        self.deconv21 = nn.DataParallel(nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=0))
        self.deconv22 = nn.ConvTranspose2d(64, 64, kernel_size=2, stride=1, padding=0)
        self.relu5 = nn.ELU()
        
        self.deconv31 = nn.DataParallel(nn.ConvTranspose2d(64, 1, kernel_size=3, stride=1, padding=0))
        self.deconv32 = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=1, padding=0)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        if self.training:
            x = self.relu1(self.pool1(self.conv1(x)))
            x = self.dropout(x)
            x = self.relu2(self.pool2(self.conv2(x)))
            x = self.dropout(x)
    #         x = self.relu3(self.pool3(self.conv32(self.conv31(x))))

    #         x = self.relu4(self.unpool1(self.deconv12(self.deconv11(x))))
            x = self.relu5(self.deconv22(self.deconv21(x)))
            x = self.dropout(x)
            x = self.deconv32(self.deconv31(x))

            x = self.sigmoid(x)
        
        else:
            x = self.relu1(self.pool1(self.conv1(x)))
            x = self.relu2(self.pool2(self.conv2(x)))
    #         x = self.relu3(self.pool3(self.conv32(self.conv31(x))))
    #         x = self.relu4(self.unpool1(self.deconv12(self.deconv11(x))))
            x = self.relu5(self.deconv22(self.deconv21(x)))
            x = self.deconv32(self.deconv31(x))

            x = self.sigmoid(x)
            
        return x

In [ ]:
# class FCN32s(nn.Module):

#     def __init__(self, pretrained_net, n_class):
#         super().__init__()
#         self.n_class = n_class
#         self.pretrained_net = pretrained_net
#         self.relu    = nn.ReLU(inplace=True)
#         self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
#         self.bn1     = nn.BatchNorm2d(512)
#         self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
#         self.bn2     = nn.BatchNorm2d(256)
#         self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
#         self.bn3     = nn.BatchNorm2d(128)
#         self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
#         self.bn4     = nn.BatchNorm2d(64)
#         self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
#         self.bn5     = nn.BatchNorm2d(32)
#         self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

#     def forward(self, x):
#         output = self.pretrained_net(x)
#         x5 = output['x5']  # size=(N, 512, x.H/32, x.W/32)
#         print(x5.shape)
#         x = self.bn1(self.relu(self.deconv1(x5)))     # size=(N, 512, x.H/16, x.W/16)
#         print(x.shape)
#         x = self.bn2(self.relu(self.deconv2(x)))  # size=(N, 256, x.H/8, x.W/8)
#         print(x.shape)
#         x = self.bn3(self.relu(self.deconv3(x)))  # size=(N, 128, x.H/4, x.W/4)
#         print(x.shape)
#         x = self.bn4(self.relu(self.deconv4(x)))  # size=(N, 64, x.H/2, x.W/2)
#         print(x.shape)
#         x = self.bn5(self.relu(self.deconv5(x)))  # size=(N, 32, x.H, x.W)
#         print(x.shape)
#         x = self.classifier(x)                   # size=(N, n_class, x.H/1, x.W/1)
#         print(x.shape)

#         return x  # size=(N, n_class, x.H/1, x.W/1)

In [6]:
def normalize_image(img):
        return img / 255
    
def image_preprocessing(img, dtype):
    if dtype == 'image':
        img = normalize_image(img).transpose(2, 0, 1)
        
    elif dtype == 'label':
        img = normalize_image(img).reshape(1, *img.shape)
        
    else:
        raise ValueError("Wrong dtype!")
    
    img = img.reshape(-1, *img.shape)
    
    return torch.from_numpy(img)
    
def dataloader(training=True):
    if training:
        x_path1 = '/home/calvinku/projects/yangzhe/data/cell01/*.png'
        y_path1 = '/home/calvinku/projects/yangzhe/data/xu_label_cell01/*.png'
        x_path2 = '/home/calvinku/projects/yangzhe/data/cell022/*.png'
        y_path2 = '/home/calvinku/projects/yangzhe/data/xu_label_cell022/*.png'

    else:
        x_path1 = '/home/calvinku/projects/yangzhe/data/test/eu_bin/*.png'
        y_path1 = '/home/calvinku/projects/yangzhe/data/test/xu_test_label/*.png'
        x_path2 = ''
        y_path2 = ''
        
    x_filenames = np.array(glob(x_path1) + glob(x_path2))
    y_filenames = np.array(glob(y_path1) + glob(y_path2))

    len_data = len(x_filenames)

    shuffle_index = np.random.choice(range(len_data), size=len_data, replace=False)

    x_filenames = x_filenames[shuffle_index]
    y_filenames = y_filenames[shuffle_index]
            
    if len(x_filenames) != len(y_filenames):
        raise ValueError("Data size doesn't match that of label.")

    for i, filename in enumerate(x_filenames):
        img = imread(filename)
        label = imread(y_filenames[i])

#         x = normalize_image(img).transpose(2, 0, 1)
#         y = normalize_image(label).reshape(1, *label.shape)

#         x, y = x.reshape(-1, *x.shape), y.reshape(-1, *y.shape)
#         x, y = torch.from_numpy(x), torch.from_numpy(y)    

#         yield x, y
        yield image_preprocessing(img, dtype='image'), image_preprocessing(label, dtype='label')
        

        

In [7]:
# x_path = '/home/calvinku/projects/yangzhe/data/cell01/'
# y_path = '/home/calvinku/projects/yangzhe/data/xu_label_cell01/'

# data_gen = dataloader(x_path, y_path)

# for i, data in enumerate(data_gen):
#     x, y = data
    
#     print(x.shape, y.shape)
    
#     if i > 10:
#         break


In [8]:
# from torchvision import models
import torch.optim as optim
from torch.autograd import Variable

if __name__ == "__main__":
    num_epochs = 10
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
#     # test output size
#     vgg_model = VGGNet(requires_grad=True)
#     input = torch.autograd.Variable(torch.randn(batch_size, 3, 224, 224))
#     output = vgg_model(input)
#     assert output['x5'].size() == torch.Size([batch_size, 512, 7, 7])

#     fcn_model = FCN32s(pretrained_net=vgg_model, n_class=n_class)
#     input = torch.autograd.Variable(torch.randn(batch_size, 3, h, w))
#     output = fcn_model(input)
#     assert output.size() == torch.Size([batch_size, n_class, h, w])

#     print("Pass size check")

#     # test a random batch, loss should decrease

#     pretrained_net = VGGNet()
#     fcn_model = FCN32s(pretrained_net=pretrained_net, n_class=1)
    fcn_model = FCN()
    fcn_model = nn.DataParallel(fcn_model).to(device)
    
    if torch.cuda.is_available():
        fcn_model.cuda()
        
#     criterion = nn.BCEWithLogitsLoss()
    criterion = nn.BCELoss()
    optimizer = optim.SGD(fcn_model.parameters(), lr=1e-3, momentum=0.9)
        
#     num_layers  = len(list(fcn_model.parameters()))

#     for i in range(num_layers):
#         print(list(fcn_model.parameters())[i].size())
        
    epic_losses = []
        
    for epoch in range(num_epochs):
        print("Epoch {}".format(epoch))
        data_gen = dataloader()
        test_data_gen = dataloader(training=False)
        
        iter_losses = []
        
        for i, data in enumerate(data_gen):
            x, y = data
            x, y = Variable(x.type(torch.FloatTensor), requires_grad=False), Variable(y.type(torch.FloatTensor), requires_grad=False)
            
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            
            optimizer.zero_grad()
            
            fcn_model.train()
            output = fcn_model(x)
                        
            loss = criterion(output, y)
            loss.backward()
            
            iter_losses.append(loss.data[0].item())
            
            if i % 200 == 0:
                print("Iteration: {}, Loss: {}".format(i, loss.data[0].item()))
                
            optimizer.step()
            
        epic_losses.append(sum(iter_losses) / len(iter_losses))
        print("Epic loss: {}".format(epic_losses[-1]))
        
#         test_losses = []
        
#         for i, test_data in enumerate(test_data_gen):
#             x, y = test_data
#             x, y = Variable(x.type(torch.FloatTensor), requires_grad=False), Variable(y.type(torch.FloatTensor), requires_grad=False)
            
#             if torch.cuda.is_available():
#                 x = x.cuda()
#                 y = y.cuda()
                        
#             fcn_model.eval()
#             output = fcn_model(x)
#             loss = criterion(output, y)
#             test_losses.append(loss.data[0].item())
            
#         print("Test loss: {}".format(sum(test_losses) / len(test_losses)))
            
        

Epoch 0


/home/calvinku/pytorch/lib/python2.7/site-packages/ipykernel_launcher.py:66: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/calvinku/pytorch/lib/python2.7/site-packages/ipykernel_launcher.py:69: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iteration: 0, Loss: 0.551888167858


RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/generic/THCStorage.cu:58

In [ ]:
# test_path = '/home/calvinku/projects/yangzhe/data/test/eu_bin/'
# test_filenames = glob(test_path + '*.png')
# test_index = np.random.choice(len(test_filenames))
# test_img = imread(test_filenames[test_index])

test_img = img

img_torch = torch.from_numpy((normalize_image(test_img).reshape(-1, *test_img.shape).transpose(0, 3, 1, 2)))
img_torch.shape
img_torch = img_torch.type(torch.FloatTensor).cuda()

seg = fcn_model(img_torch)

In [ ]:
pred = seg.cpu().detach().numpy().reshape(seg.shape[1], seg.shape[2], seg.shape[3]).transpose(1, 2, 0)
pred = pred.reshape(pred.shape[0], pred.shape[1])

In [ ]:
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0

plt.figure(figsize=(16, 9))
plt.imshow(pred)

In [ ]:
import seaborn as sns

plt.figure(figsize=(16, 9))
sns.set_style("darkgrid")
plt.plot(losses)
plt.show()
